# Strategy

## O que é?

O padrao _Strategy_ permite que objetos criados por uma classe tenham comportamentos diferentes para o mesmo método sem que o comportamento esteja atrelado à lógica da subclasse. Ele também permite que o comportamento mude durante o runtime.

## Problema a ser resolvido

Objetos da mesma classe precisam executar uma determinada acao de maneiras diferentes, mantendo a flexibilidade e evitando 

## Exemplo

Um exemplo clássico de uso do padrao Strategy seria a criacao de personagens num jogo no estilo RPG. Suponhamos que o jogo tenha personagens com as seguintes características: 

In [5]:
class Character:
    def __init__(self, name, job):
        self.name = name
        self.job = job
    
    def attack(self):
        print(f"{self.name} is killing you")


c1 = Character('Player1', 'Fighter')
c2 = Character('Player2', 'Black Mage')
c3 = Character('Player3', 'Bard')

c1.attack()
c2.attack()
c3.attack()

Player1 is killing you
Player2 is killing you
Player3 is killing you


Vamos supor que queremos implementar métodos de ataque distintos para cada job. Uma implementacao naive seria a seguinte:

In [6]:
class Character:
    def __init__(self, name, job):
        self.name = name
        self.job = job
    
    def attack(self):
        if self.job == 'Fighter':
            print(f"{self.name} is killing you with his mighty fists")
        if self.job == 'Black Mage':
            print(f"{self.name} is killing you with the power of magic")
        if self.job == 'Bard':
            print(f"{self.name} is killing you with the power of music")
            
c1 = Character('Player1', 'Fighter')
c2 = Character('Player2', 'Black Mage')
c3 = Character('Player3', 'Bard')

c1.attack()
c2.attack()
c3.attack()

Player1 is killing you with his mighty fists
Player2 is killing you with the power of magic
Player3 is killing you with the power of music


É fácil ver como a complexidade do método `attack` pode explodir com a criacao de novos jobs ou métodos de ataque. Uma maneira otimista de contornar isso seria usando inheritance e criando subclasses para cada job

In [7]:
class Character:
    def __init__(self, name, job):
        self.name = name
        self.job = job
    
    def attack(self):
        pass

class Fighter(Character):
    def __init__(self, name):
        super().__init__(name,"Fighter")
    
    def attack(self):
        print(f"{self.name} is killing you with his mighty fists")

class BlackMage(Character):
    def __init__(self, name):
        super().__init__(name,"Black Mage")
    
    def attack(self):
        print(f"{self.name} is killing you with the power of magic")

class Bard(Character):
    def __init__(self, name):
        super().__init__(name,"Bard")
    
    def attack(self):
        print(f"{self.name} is killing you with the power of music")

c1 = Fighter('Player1')
c2 = BlackMage('Player2')
c3 = Bard('Player3')

c1.attack()
c2.attack()
c3.attack()

Player1 is killing you with his mighty fists
Player2 is killing you with the power of magic
Player3 is killing you with the power of music
